# My Music 

In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

from mp3id import mp3ID
from musicBase import musicBase
from musicFinder import musicFinder
from musicPath import pathBasics
from timeUtils import clock, elapsed
from discogsBase import discogs
from multiArtist import multiArtist
from time import sleep
#from discogs import discogs

## Python Version
import sys
print("Python: {0}".format(sys.version))

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-09-28 18:59:42.829838


# Music Analyzer (My Music)

In [2]:
%load_ext autoreload
%autoreload

from searchUtils import findDirsPattern, findWalkExt
from fileUtils import getBaseFilename, getFileBasics
from fsUtils import setFile
from ioUtils import saveFile, getFile
from timeUtils import clock, elapsed
from musicPath import pathBasics


##############################################################################################################################
# My Music
##############################################################################################################################
class musicAnalyzer(musicBase):
    def __init__(self, debug=False):
        self.name = "myanalyzer"
        musicBase.__init__(self, debug=debug)
        
        self.pb      = pathBasics()
        self.flists  = self.getMusicListFiles()
        self.tagDBs  = self.getMusicTagDBs()
        self.pathDBs = self.getMusicPathDBs()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
%load_ext autoreload
%autoreload
from albums import albums
disc = discogs()
discdf = disc.getMasterDiscogsDB()
print(discdf.shape)
discdf = discdf[~discdf['Name'].isna()]
print(discdf.shape)

artists = list(discdf["Artist"])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Local Discog Directory /Users/tgadfort/Music/Discog is Available
/Volumes/Music/Discog/base exists
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/base-db exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-extra exists
/Volumes/Music/Discog/search exists
/Volumes/Music/Discog/search-artists exists
/Volumes/Music/Discog/special exists
/Volumes/Music/Discog/artist-special exists
/Volumes/Music/Discog/db exists
Found 100 artist DB files and that is equal to the max mod value
Loading data from /Volumes/Music/Discog/db/MasterDB.p
  --> This file is 57.6MB.
Loading /Volumes/Music/Discog/db/MasterDB.p
(480573, 8)
(480526, 8)


In [91]:
%load_ext autoreload
%autoreload
mf = musicFinder(debug=False)
## Run This To Find Music Paths
mf.findMusic()
ma = musicAnalyzer(debug=False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Checking /Volumes/Music/iTunes Classical...   	Found 3414 files.
Processed 3414 paths.
Saving data to /Volumes/Music/MusicDB/Music-iTunes Classical.p
  --> This file is 154.6kB.
Saved data to /Volumes/Music/MusicDB/Music-iTunes Classical.p
  --> This file is 154.6kB.
Checking /Volumes/Music/iTunes Compilation...   	Found 28335 files.
Processed 28335 paths.
Saving data to /Volumes/Music/MusicDB/Music-iTunes Compilation.p
  --> This file is 1.5MB.
Saved data to /Volumes/Music/MusicDB/Music-iTunes Compilation.p
  --> This file is 1.5MB.
Checking /Volumes/Music/iTunes Consolidate...   	Found 61459 files.
Processed 61459 paths.
Saving data to /Volumes/Music/MusicDB/Music-iTunes Consolidate.p
  --> This file is 2.9MB.
Saved data to /Volumes/Music/MusicDB/Music-iTunes Consolidate.p
  --> This file is 2.9MB.
Checking /Volumes/Music/iTunes Jazz...   	Found 3747 files.
Processed 3747 paths.
Saving data to /Vo

In [92]:
start, cmt = clock("Getting My Artists/Albums")

artistData = {}
albumData = {}
for pathfile in ma.pathDBs:
    if sum([x in pathfile for x in ["Soundtrack", "MixTape", "Compilation", "Non Fiction", "Todo", "Moves", "Not In Discogs"]]) > 0:
        continue
    if sum([x in pathfile for x in ["Consolidate"]]) == 0:
        continue
    pathdata = getFile(pathfile)
    for mp3,mp3data in pathdata.items():
        if artistData.get(mp3data['pbArtist']) is None:
            artistData[mp3data["pbArtist"]] = {}
        if artistData[mp3data["pbArtist"]].get(mp3data["pbAlbum"]) is None:
            artistData[mp3data["pbArtist"]][mp3data["pbAlbum"]] = []
        artistData[mp3data["pbArtist"]][mp3data["pbAlbum"]].append(mp3)

        if albumData.get(mp3data["pbAlbum"]) is None:
            albumData[mp3data["pbAlbum"]] = {}
        if albumData[mp3data["pbAlbum"]].get(mp3data["pbArtist"]) is None:
            albumData[mp3data["pbAlbum"]][mp3data["pbArtist"]] = []
        albumData[mp3data["pbAlbum"]][mp3data["pbArtist"]].append(mp3)
        
    print("Found {0} artists and {1} albums".format(len(artistData), len(albumData)))
    
    
saveFile(idata=artistData, ifile="/Volumes/Music/MusicDB/artistData.p")
saveFile(idata=albumData, ifile="/Volumes/Music/MusicDB/albumData.p")

elapsed(start, cmt)

Current Time is Mon Sep 30, 2019 18:12:59 for Getting My Artists/Albums
Found 2488 artists and 5896 albums
  --> This file is 1.4MB.
  --> This file is 1.4MB.
Current Time is Mon Sep 30, 2019 18:13:09 for Done with Getting My Artists/Albums
Process [{0}] took 9 seconds.


In [6]:
if False:
    for albumID,albumVals in albumData.items():
        if len(albumVals) > 1:
            print(albumID,'\t',end="\t")
            print({k: len(v) for k,v in albumVals.items()})
            print("")

# Find Artists

In [7]:
#discdf[discdf['Artist'].str.contains("Adel")]

In [ ]:
from searchUtils import findNearest
from unicodedata import normalize
from pandas import DataFrame

def getMusicData(key, artist):
    retval = discdf[discdf[key] == artist]
    if retval.shape[0] > 0:
        return retval
    else:
        return None


saveArtistData = {"Odd": {}, "Match": {}, "Near": {}, "None": {}}

for artistName,artistAlbums in artistData.items():
    
    rename = None
    if False:
        rename = None
        if renameReplacements.get(artistName) is not None:
            artistName = renameReplacements[artistName]
            rename = True
        
    
    artistName = normalize('NFC', artistName)

    
    keys = ["Artist", "Name"]
    done = False
    for key in keys:
        mdata = getMusicData(key, artistName)
        if isinstance(mdata, DataFrame):
            if key == "Artist":
                #print('Match\t',mdata.shape,'\t',artistName)
                saveArtistData["Match"][artistName] = {"Match": artistName, "Rename": rename, "Discog": mdata, "Albums": artistAlbums}
            else:
                print('Odd\t',mdata.shape,'\t',artistName)
                saveArtistData["Odd"][artistName] = {"Match": artistName, "Rename": rename, "Discog": mdata, "Albums": artistAlbums}
            done = True
            break
    if done:
        continue

    cutoff=0.9
    if len(artistName) <= 6:
        cutoff = 0.8
    cutoffArtists = findNearest(item=artistName, ilist=artists, num=1, cutoff=cutoff)
    if len(cutoffArtists) > 0:
        for artist in cutoffArtists:
            mdata = getMusicData("Artist", artist)
            saveArtistData["Near"][artistName] = {"Match": artist, "Rename": rename, "Discog": mdata, "Albums": artistAlbums}
            print('Near','\t',mdata.shape,'\t',artistName,'\t\t====> Discog Match =',artist)
            done = True
    else:
        if False:
            cutoffArtists = findNearest(item=artistName, ilist=artists, num=1, cutoff=cutoff - 0.15)
            if len(cutoffArtists) > 0:
                for artist in cutoffArtists:
                    mdata = getMusicData("Artist", artist)
                    saveArtistData["Near"][artistName] = {"Match": artist, "Rename": rename, "Discog": mdata, "Albums": artistAlbums}
                    print('Near','\t',mdata.shape,'\t',artistName,'\t\t====> Discog Match =',artist)
                    done = True

        
    if done:
        continue
        
    print('\t','\t','\t',artistName)
    saveArtistData["None"][artistName] = {"Match": None, "Rename": rename, "Discog": None, "Albums": artistAlbums}
    continue
    
    
    if artistName == "Anorexia Nervosa (2)":
        print(artistAlbums)
        1/0
    continue
    for albumName,albumVals in artistAlbums.items():
        print("\t",albumName,len(albumVals))    
    break
#albumData = {}

	 	 	  ...And Oceans
	 	 	 2Pac & Outlawz
	 	 	 3rd House
	 	 	 Aaron Neville & Celine Dion
Near 	 (1, 8) 	 ACDC 		====> Discog Match = ACxDC
	 	 	 Ali Vegas & L-Gee
	 	 	 Andreas Ammer & FM Einheit & Ulrike Haage
Odd	 (1, 8) 	 Avenger (6)
	 	 	 B.B. King & Eric Clapton
	 	 	 B.B. King & Mary Travers
	 	 	 B.K.S..S
	 	 	 Bad Seeds & Zakary Thaks
	 	 	 Barry White & Love Unlimited
	 	 	 Benny Andersen & Poul Dissing
	 	 	 Billy Joel & Richard Joo
	 	 	 Birdman & Rick Ross
	 	 	 Black Bill Gates & DJ Spinz
	 	 	 Bluegrass 2001
	 	 	 Blues Brothers
	 	 	 Blumchen
	 	 	 Bob Dylan & Johnny Cash
	 	 	 Bob Kogut & Dana Kogut
	 	 	 Bobby Osborne & Jesse McReynolds
	 	 	 Bobby Pickett + Crypt Kickers
	 	 	 Brooklyn Heights
	 	 	 Buddy Guy & Jonny Lang
	 	 	 Busta Rhymes & Reek Da Villain & J Doe
	 	 	 Butch Baldassari & John Carlini
	 	 	 Cam'ron & Juelz Santana
	 	 	 Cam'ron & Mike Prat
	 	 	 Cam'ron & Vado
	 	 	 Camron Silver
	 	 	 Capital 2
	 	 	 Charlie Haden & Hank Jones
	 	 	 Charlie Wall

# Known Discog Renames

In [ ]:
from pandas import read_excel
renames = read_excel('/Volumes/Music/Discog/db/DiscogRenames.xlsx')[["Unnamed: 1", "Unnamed: 2"]][1:]
renames.columns = ["My Name", "Discog Name"]
renames

In [ ]:
cutoff=0.8
for discogName in renames["Discog Name"].values:
    cutoffArtists = findNearest(item=discogName, ilist=artists, num=1, cutoff=cutoff)
    print(discogName,'\t',cutoffArtists)

In [ ]:
renames["Discog Name"].values

# Mark Soundtracks

In [ ]:
from searchUtils import findDirsPattern, findAll
from fsUtils import moveDir
from fileUtils import getDirBasics
for dirval in findDirsPattern('/Volumes/Music/iTunes Compilation/iTunes Media/Music', pattern="OST"):
    if dirval.endswith("OST"):
        dst = " ".join([dirval[:-3].strip(), "Soundtrack"])
        albumName = " ".join([getDirBasics(dirval)[-1][:-3].strip(), "Soundtrack"])
        for val in findAll(dirval):
            for albumFile in findAll(val):
                m = mp3ID(albumFile)
                m.setAlbum(albumName)
                
        moveDir(dirval, dst)

# Near Matches

In [ ]:
toExcel = []
for i,(artistName,aData) in enumerate(saveArtistData["Near"].items()):
    print("{0: <3}/{1: <3}".format(i,len(saveArtistData["Near"])), end=" \t")
    rename = aData["Rename"]
    discog = aData["Discog"]
    albums = aData["Albums"]
    if discog is not None:
        matches = discog["Name"].values
        print('{0: <50}'.format(artistName),matches)
    else:
        print('{0: <50}{1: <8}{2}'.format(artistName,'x',len(albums)))

# Odd Names

In [83]:
for artistName,aData in saveArtistData["Odd"].items():
    rename = aData["Rename"]
    discog = aData["Discog"]
    albums = aData["Albums"]
    if discog is not None:
        matches = discog["Name"].values
        
        print('{0: <50}'.format(artistName),matches)
    else:
        print('{0: <50}{1: <8}{2}'.format(artistName,'x',len(albums)))
        

Avenger (6)                                        ['Avenger (6)']
Brand X (3)                                        ['Brand X (3)']
Chris Lee (28)                                     ['Chris Lee (28)']
D.O.A. (2)                                         ['D.O.A. (2)']
Daddy Longlegs (6)                                 ['Daddy Longlegs (6)']
De Profundis (5)                                   ['De Profundis (5)']
Despair (3)                                        ['Despair (3)']
Diamond Head (2)                                   ['Diamond Head (2)']
Eightball (3)                                      ['Eightball (3)']
Endless Struggle (2)                               ['Endless Struggle (2)']
Eyeball (4)                                        ['Eyeball (4)']
Fancie (2)                                         ['Fancie (2)']
Forgotten (3)                                      ['Forgotten (3)']
Glimmer (2)                                        ['Glimmer (2)']
Homeless (5)                   

# None Match

In [10]:
## Abwaerts = Abwärts
## Move all ' vs ' into MixTape

## Known
#Beatsnblends
#Dr. Tom Butt

In [13]:
artistData['Otis Spann & FM']

{'The Biggest Thing Since Colossus': ['/Volumes/Music/iTunes Matched/iTunes Media/Music/Otis Spann & FM/The Biggest Thing Since Colossus/01 My Love Depends On You.mp3',
  "/Volumes/Music/iTunes Matched/iTunes Media/Music/Otis Spann & FM/The Biggest Thing Since Colossus/02 Walkin'.mp3",
  '/Volumes/Music/iTunes Matched/iTunes Media/Music/Otis Spann & FM/The Biggest Thing Since Colossus/03 It Was A Big Thing.mp3',
  '/Volumes/Music/iTunes Matched/iTunes Media/Music/Otis Spann & FM/The Biggest Thing Since Colossus/04 Temperature Is Rising (100.2 dF).mp3',
  '/Volumes/Music/iTunes Matched/iTunes Media/Music/Otis Spann & FM/The Biggest Thing Since Colossus/05 Dig You.mp3',
  "/Volumes/Music/iTunes Matched/iTunes Media/Music/Otis Spann & FM/The Biggest Thing Since Colossus/06 No More Doggin'.mp3",
  "/Volumes/Music/iTunes Matched/iTunes Media/Music/Otis Spann & FM/The Biggest Thing Since Colossus/07 Ain't Nobody's Business.mp3",
  '/Volumes/Music/iTunes Matched/iTunes Media/Music/Otis Spann 

In [84]:
from searchUtils import findNearest

class multiArtist:
    def __init__(self, cutoff=0.9, discdata=None, exact=False):
        self.cutoff   = cutoff    
        self.discdata = discdata
        self.exact    = exact
        
        self.basicdelims = ["Duet With", "Presents", "Featuring"]
        self.delims = [",", "&", " And ", "+", "/", "With The", " with ", " With ", " y ", " Y ", " feat.",  " ft.", " Feat. ", " x ", " X "]
        self.discArtists = []
        if self.discdata is not None:
            self.discArtists = [x for x in discdata.keys() if x is not None]
        self.knownDelimArtists = {artist: True for artist in self.discArtists if self.nDelims(artist) > 0}

    
    def getDiscArtists(self):
        return self.discArtists
    
    def getKnownDelimArtists(self):
        return self.knownDelimArtists  
    
    def isKnownArtist(self, artist):
        if isinstance(self.discdata, dict):
            return self.discdata.get(artist) != None
        return False
    
    def nDelims(self, artist):
        return sum([x in artist for x in self.delims])
    
    
    def joinNames(self, delim, names):
        joinVal = delim
        if delim.startswith(" ") is False:
            joinVal = " {0}".format(joinVal)
        if delim.endswith(" ") is False:
            joinVal = "{0} ".format(joinVal)
        
        return joinVal.join(names)
    
    
    def getDelimData(self, val, debug=False):
        valdata = {d: [x.strip() for x in val.split(d)] for d in self.delims if d in val}
        if debug:
            print("\tDelimParsing ==> {0}".format(valdata))
        for k,v in valdata.items():
            if len(v) == 3:
                v.append(self.joinNames(delim=k, names=v[:2]))
                v.append(self.joinNames(delim=k, names=v[1:-1]))
        if debug:
            print("\tDelimParsing ==> {0}".format(valdata))

        return valdata
    
    
    def getBasicDelimData(self, val):
        valdata = {d: [x.strip() for x in val.split(d)] for d in self.basicdelims if d in val}
        return valdata

    def getNdelims(self, val):
        return len(val)

    def addArtist(self, allArtists, val, debug=False):
        #print("L={0}".format(len(allArtists)))
        if allArtists.get(val) is None:
            if debug:
                print("Adding {0}. Sum is {1}".format(val, len(allArtists)))
            allArtists[val] = True
    
    
    def cleanArtist(self, artist):
        artist = artist.replace("(", "")
        artist = artist.replace(")", "")
        return artist


    def newMethod(self, artist, debug=False):
        if debug is True:
            print("Parsing [{0}]".format(artist))
        artist = self.cleanArtist(artist)
        allArtists = {}
        d1delims = self.getBasicDelimData(artist)
        if len(d1delims) == 0:
            d1delims = self.getDelimData(artist)
        if debug:
            print('1','\t',artist,'===>',d1delims)

        knownArtists = set()
        if len(d1delims) == 0:
            self.addArtist(allArtists, artist, debug)
            knownArtists = set(allArtists.keys())
        elif self.isKnownArtist(artist):
            self.addArtist(allArtists, artist, debug)
            knownArtists = set(allArtists.keys())
            d1delims = {}


        ##############################################################################
        ## 1st Delimiter Split
        ##############################################################################
        for delim1, delimdata1 in d1delims.items():
            delimdata1 = list(set(delimdata1).difference(knownArtists))
            for artist1 in delimdata1:
                d2delims = self.getDelimData(artist1)
                if debug:
                    print('2','\t',artist1,'===>',d2delims)
                if self.getNdelims(d2delims) == 0:
                    self.addArtist(allArtists, artist1, debug)
                    knownArtists = set(allArtists.keys())
                    continue
                elif self.isKnownArtist(artist1):
                    self.addArtist(allArtists, artist1, debug)
                    knownArtists = set(allArtists.keys())
                    d2delims = {}

                    
                ##############################################################################
                ## 2nd Delimiter Split
                ##############################################################################
                for delim2, delimdata2 in d2delims.items():
                    delimdata2 = list(set(delimdata2).difference(knownArtists))
                    for artist2 in delimdata2:
                        d3delims = self.getDelimData(artist2)
                        if debug:
                            print('3','\t',artist2,'===>',d3delims)
                        if self.getNdelims(d3delims) == 0:
                            self.addArtist(allArtists, artist2)
                            knownArtists = set(allArtists.keys())
                            continue
                        elif self.isKnownArtist(artist2):
                            self.addArtist(allArtists, artist2)
                            knownArtists = set(allArtists.keys())
                            d3delims = {}

                            
                        ##############################################################################
                        ## 3rd Delimiter Split
                        ##############################################################################
                        for delim3, delimdata3 in d3delims.items():
                            delimdata3 = list(set(delimdata3).difference(knownArtists))
                            for artist3 in delimdata3:
                                d4delims = self.getDelimData(artist3)
                                if self.getNdelims(d4delims) == 0:
                                    self.addArtist(allArtists, artist3)
                                    knownArtists = set(allArtists.keys())
                                    continue
                                elif self.isKnownArtist(artist3):
                                    self.addArtist(allArtists, artist3)
                                    knownArtists = set(allArtists.keys())
                                    d4delims = {}
                                    

                                ##############################################################################
                                ## 4th Delimiter Split
                                ##############################################################################
                                for delim4, delimdata4 in d4delims.items():
                                    delimdata4 = list(set(delimdata4).difference(knownArtists))
                                    for artist4 in delimdata4:
                                        d5delims = self.getDelimData(artist4)
                                        if self.getNdelims(d5delims) == 0:
                                            self.addArtist(allArtists, artist4)
                                            knownArtists = set(allArtists.keys())
                                            continue
                                        elif self.isKnownArtist(artist4):
                                            self.addArtist(allArtists, artist4)
                                            knownArtists = set(allArtists.keys())
                                            d5delims = {}


                                        ##############################################################################
                                        ## 5th Delimiter Split
                                        ##############################################################################
                                        for delim5, delimdata5 in d5delims.items():
                                            delimdata5 = list(set(delimdata5).difference(knownArtists))
                                            for artist5 in delimdata5:
                                                d6delims = self.getDelimData(artist5)
                                                if self.getNdelims(d6delims) == 0:
                                                    self.addArtist(allArtists, artist5)
                                                    knownArtists = set(allArtists.keys())
                                                    continue
                                                elif self.isKnownArtist(artist5):
                                                    self.addArtist(allArtists, artist5)
                                                    knownArtists = set(allArtists.keys())
                                                    d6delims = {}

                                                    

        ##############################################################################
        ## Combine Results
        ##############################################################################                                                    
        results = {}
        if self.discdata is not None and len(self.discArtists) > 0:
            for name in knownArtists:
                retval = self.discdata.get(name)
                if self.exact is False:
                    if retval is None:
                        retval = findNearest(name, self.discArtists, 1, self.cutoff)
                        if len(retval) == 1:
                            retval = self.discdata.get(retval[0])
                        else:
                            retval = None
                        
                results[name] = retval
        else:
            results = {k: ['?'] for k,v in allArtists.items()}
                
        return results

    def getArtistNames(self, artist, debug=False):        
        return self.newMethod(artist, debug)
    
    
    
        if self.nDelims(artist) == 0:
            names = {artist: []}
            return names
        
        if self.discdata is not None and len(self.discArtists) > 0:
            retval = self.discdata.get(artist)
            if retval is not None:
                return {artist: retval}
            else:
                retval = findNearest(artist, self.discArtists, 1, self.cutoff)
                if len(retval) == 1:
                    return {artist: self.discdata.get(retval[0])}        
        
        names = {artist: None}
        names = self.splitArtist(names)
        names = self.unravelDict(names)
        names = self.unravelDict(names)
        return names


    def unravelDict(self, dvals):
        fvals = {}
        for k,v in dvals.items():
            if isinstance(v, dict):
                for k2,v2 in v.items():
                    if isinstance(v2, dict):
                        for k3,v3 in v.items():
                            if isinstance(v3, dict):
                                for k4,v4 in v.items():
                                    fvals[k4] = v4
                            else:
                                fvals[k3] = v3
                    else:
                        fvals[k2] = v2
            else:
                fvals[k] = v

        return fvals


    def splitArtistDelim(self, artist, delval):
        names = {}    
        if delval not in artist:
            return None

        for val in artist.split(delval):
            val = val.strip()
            
            if self.discdata is not None and len(self.discArtists) > 0:
                retval = self.discdata.get(val)
                if retval is not None:
                    names[val] = retval
                else:
                    retval = findNearest(val, self.discArtists, 1, self.cutoff)
                    if len(retval) == 0:
                        names[val] = None
                    else:
                        names[val] = retval
            else:
                names[val] = [-1]

        if len(names) == 0:
            return None

        if any(names.values()) is False:
            return None

        return names


    def splitArtist(self, result):
        delims = self.delims
            
        #print("Input: {0}".format(result))
        for name in result.keys():
            #print("  Name -->",name,"\tCurrent Value -->",result[name])
            if result[name] is None:
                for delim in delims:
                    #print("\tDelim: {0}  for {1}".format(delim, name))
                    result2 = self.splitArtistDelim(name,delval=delim)
                    #print("\tDelim Result: {0}".format(result2))


                    if result2 is not None:
                        result[name] = result2
                        #print("\tName:",name,'\tResult:',result2)
                        for name2 in result2.keys():
                            if result2[name2] is None:
                                for delim2 in delims:
                                    #print("\t\tDelim2: {0}  for {1}".format(delim2, name2))
                                    result3 = self.splitArtistDelim(name2,delval=delim2)
                                    #print("\t\tDelim Result: {0}".format(result3))


                                    if result3 is not None:
                                        #print("\t\tName:",name2,'\tResult:',result3)
                                        result2[name2] = result3

                                        ## Breaking from delim2 
                                        break

                        ## Breaking from delim
                        break

        return result

In [11]:
#artistDB = disc.getArtistNameToIDData()
artistNameToID = {}
for artistID, artistName in discdf["Artist"].to_dict().items():
    if artistNameToID.get(artistName) is None:
        artistNameToID[artistName] = []
    artistNameToID[artistName].append(artistID)

In [85]:
mulArts  = multiArtist(cutoff=1.0, discdata=artistNameToID, exact=False)

# 

In [24]:
v = ['Belle', 'Sebastian', 'Norah Jones']

In [89]:
albumData["live and rare"]

{'Brian Setzer': ['/Volumes/Music/iTunes Consolidate/iTunes Media/Music/Brian Setzer/live and rare/06 Guiter Hero.mp3',
  '/Volumes/Music/iTunes Consolidate/iTunes Media/Music/Brian Setzer/live and rare/08 Everybody Wants To Be A Cat.mp3'],
 'Brian Setzer & Brian Wilson': ['/Volumes/Music/iTunes Consolidate/iTunes Media/Music/Brian Setzer & Brian Wilson/live and rare/10 Little Deuce Coupe.mp3'],
 'Brian Setzer Orchestra': ['/Volumes/Music/iTunes Consolidate/iTunes Media/Music/Brian Setzer Orchestra/live and rare/01 Dragnet Theme (Live).mp3',
  "/Volumes/Music/iTunes Consolidate/iTunes Media/Music/Brian Setzer Orchestra/live and rare/02 Rock This Town (Woodstock '99).mp3",
  '/Volumes/Music/iTunes Consolidate/iTunes Media/Music/Brian Setzer Orchestra/live and rare/03 Malaguena & Brand New Cadillac (Live).mp3',
  '/Volumes/Music/iTunes Consolidate/iTunes Media/Music/Brian Setzer Orchestra/live and rare/04 Stray Cat Strut (Live).mp3',
  '/Volumes/Music/iTunes Consolidate/iTunes Media/Musi

In [60]:
artistName = 'Belle & Sebastian & Norah Jones'
artistName = "The Ne'er Do Wells & Judy And The Loadies"
matches = mulArts.getArtistNames(artistName, debug=False)
matches

{'Judy And The Loadies': ['276320'],
 'Judy': ['3359551', '1845777', '579281'],
 "The Ne'er Do Wells": ['818820'],
 'The Loadies': None}

In [86]:
for i,(artistName,aData) in enumerate(saveArtistData["None"].items()):
    rename = aData["Rename"]
    discog = aData["Discog"]
    albums = aData["Albums"]

    matches = mulArts.getArtistNames(artistName)
    if all(matches.values()):
        for albumName,albumValueData in albums.items():
            if len(albumValueData) == 1:
                print('{0: <50}'.format(artistName),matches)
                print("\t{0}\t{1}\n".format(len(albumValueData), albumName))

Ariana Grande & The Weeknd                         {'The Weeknd': ['2171152'], 'Ariana Grande': ['3310737']}
	1	My Everything (Target Exclusive Edition)

B.B. King & Mary Travers                           {'B.B. King': ['37729'], 'Mary Travers': ['498742']}
	1	PP M& (Lifelines)

Barry White & Love Unlimited                       {'Barry White': ['1426501', '18978'], 'Love Unlimited': ['164269']}
	1	Random

Billy Ray Cyrus & Miley Cyrus                      {'Miley Cyrus': ['676432'], 'Billy Ray Cyrus': ['300597']}
	1	Random

Brian Setzer & Brian Wilson                        {'Brian Setzer': ['269949'], 'Brian Wilson': ['189718']}
	1	live and rare

Buddy Guy & Jonny Lang                             {'Buddy Guy': ['259799'], 'Jonny Lang': ['343201']}
	1	Random

Céline Dion & Aaron Neville                        {'Aaron Neville': ['15261'], 'Céline Dion': ['19298']}
	1	Random

Chris Brown & Hurricane Chris & Bow Wow            {'Chris Brown': ['4026500', '3578012', '5369712', '5017915', 

Steve Wariner & Nitty Gritty Dirt Band             {'Steve Wariner': ['692580'], 'Nitty Gritty Dirt Band': ['178818']}
	1	Miscellaneous Live Recordings

The Cardigans & Tom Jones                          {'Tom Jones': ['2269038', '51039', '782651', '1795093'], 'The Cardigans': ['39900', '1114834']}
	1	http:--questers.musicpage.com

The KLF & Tammy Wynette                            {'Tammy Wynette': ['16406'], 'The KLF': ['2666675', '3178']}
	1	Random

The Saturdays & Flo Rida                           {'Flo Rida': ['885103'], 'The Saturdays': ['1232437', '5761270']}
	1	Flo.Rida-Only.One.Flo.Pt.1-(Retail)-2010-NoFS)

Timbaland & Nelly Furtado & Justin Timberlake      {'Nelly Furtado': ['21731'], 'Timbaland': ['35125'], 'Justin Timberlake': ['66824']}
	1	Jams

Twin & Johan S                                     {'Twin': ['585300', '52205'], 'Johan S': ['2117222', '625153', '3003371']}
	1	DJ008® CD 92

Twista & DJ Victoriouz                             {'DJ Victoriouz': ['2809825'], 'Twis

In [15]:
#discdf[discdf.index == '730612']
discdf[discdf.index == '135946']

,Ref,Name,Known,Artists,Genre,Style,Artist,Artist Num
135946,/artist/135946-Johnny-Cash,Johnny Cash,True,"[Johnny Cash, Jerry Lee Lewis, Willie Nelson]","[Folk, World, & Country, Rock, Blues]","[Country, , Country Rock]",Johnny Cash,NaN


In [74]:
for artistName,aData in saveArtistData["None"].items():
    rename = aData["Rename"]
    discog = aData["Discog"]
    albums = aData["Albums"]

    matches = mulArts.getArtistNames(artistName)
    if not all(matches.values()):
        print('{0: <50}'.format(artistName),matches)

Duke Ellington with the Ron Collier Orchestra      {'Duke Ellington': ['145257'], 'the Ron Collier Orchestra': None}


In [ ]:
x = artistNames[4]
items = artistNames
my_str_as_bytes = str.encode(x, 'utf-8')
print(my_str_as_bytes,type(my_str_as_bytes)) # ensure it is byte representation


from unicodedata import normalize
keys = ['NFC', 'NFKC', 'NFD', 'NFKD']
for key in keys:
    print(x,'<---',key)    
    print(findNearest(item=x, ilist=items, num=1, cutoff=1.0))
    my_str = normalize(key, x)
    print(findNearest(item=my_str, ilist=items, num=1, cutoff=1.0))
    print(my_str,type(my_str))
    my_str_as_bytes = str.encode(my_str, 'utf-8')
    print(my_str_as_bytes,type(my_str_as_bytes)) # ensure it is byte representation
    my_decoded_str = my_str_as_bytes.decode()
    print(my_decoded_str,type(my_decoded_str)) # ensure it is string representation
    print(findNearest(item=my_decoded_str, ilist=items, num=1, cutoff=1.0))
    print("")
    

# Test

In [ ]:
ifile = "/Volumes/Music/iTunes MixTape/iTunes Media/Music/24 Shots/01 Intro (Power Of The Dollar).mp3"
isFile(ifile)

In [ ]:
x = artistNames[4]
x = 'Henryk Górecki'


items = artistNames

from unicodedata import normalize
keys = ['NFC', 'NFKC', 'NFD', 'NFKD']
for key in keys:
    print(x,'<---',key)    
    print(findNearest(item=x, ilist=items, num=1, cutoff=1.0))
    my_str = normalize(key, x)
    print(findNearest(item=my_str, ilist=items, num=1, cutoff=1.0))
    print(my_str,type(my_str))
    my_str_as_bytes = str.encode(my_str, 'utf-8')
    print(my_str_as_bytes,type(my_str_as_bytes)) # ensure it is byte representation
    my_decoded_str = my_str_as_bytes.decode()
    print(my_decoded_str,type(my_decoded_str)) # ensure it is string representation
    print(findNearest(item=my_decoded_str, ilist=items, num=1, cutoff=1.0))
    print("")
    

In [ ]:
m = mp3ID(ifile)
m.getInfo()

In [ ]:
pb  = pathBasics(debug=False)
pbc = pb.getPaths(ifile)
pbc.getDict()

In [ ]:
from mp3id import mp3ID
mp3Vals = {}
for artistName,aData in saveArtistData['Odd'].items():
    print(artistName)
    discog = aData['Discog']
    artist = None
    if isinstance(discog, DataFrame):
        if discog.shape[0] == 1:
            artist = discog['Artist'][0] #.values
            
    if artist is None:
        continue
        
    for album,albumFiles in aData['Albums'].items():
        print('  ',album)
        for albumFile in albumFiles:
            print('\t',albumFile)
            m = mp3ID(albumFile)
            mArtist = m.getArtist()[0]
            if mArtist == artistName:
                print("\t\tArtist: {0} <--> {1}".format(mArtist,artist))
                m.setArtist(artist)

            mAlbumArtist = m.getAlbumArtist()[0]
            if mAlbumArtist == artistName:
                print("\t\tAlbumArtist: {0} <--> {1}".format(mAlbumArtist,artist))
                m.setAlbumArtist(artist)
            #mp3Vals[albumFile] = mid.getInfo()
            
        



In [ ]:
mp3Vals

# Multi Artists

In [ ]:
artistDB = disc.getArtistNameToIDData()
mulArts  = multiArtist(cutoff=0.9, discdata=artistDB, exact=False)

In [ ]:
from pandas import read_excel

In [ ]:
renameReplacements

In [ ]:
findNearest(item="Andre Previn", ilist=artistDB.keys(), num=10, cutoff=0.7)

In [ ]:
x = "Céline Dion"

from unicodedata import normalize
keys = ['NFC', 'NFKC', 'NFD', 'NFKD']
for key in keys:
    continue
    print(x,'<---',key)    
    print(findNearest(item=x, ilist=items, num=1, cutoff=1.0))
    my_str = normalize(key, x)
    print(findNearest(item=my_str, ilist=items, num=1, cutoff=1.0))
    print(my_str,type(my_str))
    my_str_as_bytes = str.encode(my_str, 'utf-8')
    print(my_str_as_bytes,type(my_str_as_bytes)) # ensure it is byte representation
    my_decoded_str = my_str_as_bytes.decode()
    print(my_decoded_str,type(my_decoded_str)) # ensure it is string representation
    print(findNearest(item=my_decoded_str, ilist=items, num=1, cutoff=1.0))
    print("")
    

In [ ]:
for artistName,artistData in saveArtistData.items():
    found = artistData[0]
    if found is False:
        retval = ma.getArtistNames(artistName)
        if len(retval) > 1:
            artistNames = [k for k,v in retval.items() if v is not None]
            print("{0: <50}{1}".format(artistName,artistNames))
        else:
            print("{0: <50}{1}".format(artistName,artistData[0]))

In [ ]:
lvals = list(discdf["Artist"])
test  = "*NSYNC"
from searchUtils import findNearest
findNearest(item=test, ilist=lvals, num=1, cutoff=0.9)

In [ ]:
def getMusicData(artist):
    retval = musicdf[musicdf["Artist"] == artist]
    if retval.shape[0] > 0:
        return retval
    else:
        return None

In [ ]:
artistIDtoName = disc.getArtistIDToNameData()
artistIDtoRefs  = disc.getArtistIDToRefData()
albumIDtoName = disc.getAlbumIDToNameData()
albumIDtoRef   = disc.getAlbumIDToRefData()

artistIDtoAlbumNames = disc.getArtistIDAlbumNames()
artistIDtoAlbumIDs = disc.getArtistIDAlbumIDs()

artistMetaData = disc.getAlbumArtistMetaData()

In [ ]:
colArtistIDtoName = disc.getCollectionIDToNameData()
colArtistIDtoRefs = disc.getCollectionIDToRefData()
colArtistReftoCnts = disc.getCollectionRefCountsData()

artistIDtoName = disc.getArtistIDToNameData()
artistIDtoRefs  = disc.getArtistIDToRefData()
albumIDtoName = disc.getAlbumIDToNameData()
albumIDtoRef   = disc.getAlbumIDToRefData()

artistIDtoAlbumNames = disc.getArtistIDAlbumNames()
artistIDtoAlbumIDs = disc.getArtistIDAlbumIDs()

artistMetaData = disc.getAlbumArtistMetaData()



def splitMetaData(x):
    retval = {}
    if isinstance(x, dict):
        for k,v in x.items():
            retval[k] = [z[0] for z in v.most_common(3)]
    else:
        retval = None
    return retval

from pandas import Series, DataFrame
sArtistToRef  = Series(artistIDtoRefs)
sArtistToName = Series(artistIDtoName)
sAlbumToRef   = Series(albumIDtoRef)
sAlbumToName  = Series(albumIDtoName)

sArtistToAlbums = Series(artistIDtoAlbumIDs)
sArtistToAlbumNames = Series(artistIDtoAlbumNames)
sArtistAlbums = Series([dict(zip(x, y)) for x,y in list(zip(sArtistToAlbums.values, sArtistToAlbumNames))], index=sArtistToAlbums.index)

sArtistMetaData = Series(artistMetaData)
sArtistMetaData = sArtistMetaData.apply(splitMetaData)


cols = ["Ref"]
discdf = DataFrame(sArtistToRef)
discdf.columns = cols
discdf = discdf.join(DataFrame(sArtistToName))
cols += ["Name"]
discdf.columns = cols
tmp = DataFrame(DataFrame(sArtistMetaData)[0].tolist())
tmp.index = sArtistMetaData.index
discdf = discdf.join(tmp)
cols += ["Extra Artists", "Genres", "Styles"]
discdf.columns = cols
discdf = discdf.join(DataFrame(sArtistAlbums))
cols += ["Albums"]
discdf.columns = cols




sColArtistToRef  = Series(colArtistIDtoRefs)
sColArtistToName = Series(colArtistIDtoName)
sColArtistRefToCnts = Series(colArtistReftoCnts)

cols = ["Ref"]
coldiscdf = DataFrame(sColArtistToRef)
coldiscdf.columns = cols
coldiscdf = coldiscdf.join(DataFrame(sColArtistToName))
cols += ["Name"]
coldiscdf.columns = cols

colrefdf = DataFrame(sColArtistRefToCnts)
colrefdf.columns = ["Counts"]
colrefdf.reset_index(inplace=True)
colrefdf.columns = ["Ref", "Counts"]
coldiscdf = coldiscdf.merge(colrefdf, on="Ref")
coldiscdf.index = DataFrame(sColArtistToRef).index

coldiscdf = coldiscdf.merge(discdf, on=["Ref", "Name"], how='left')
coldiscdf.index = DataFrame(sColArtistToRef).index

In [ ]:
sColArtistToRef  = Series(colArtistIDtoRefs)
sColArtistToName = Series(colArtistIDtoName)
sColArtistRefToCnts = Series(colArtistReftoCnts)

cols = ["Ref"]
coldiscdf = DataFrame(sColArtistToRef)
coldiscdf.columns = cols
coldiscdf = coldiscdf.join(DataFrame(sColArtistToName))
cols += ["Name"]
coldiscdf.columns = cols

colrefdf = DataFrame(sColArtistRefToCnts)
colrefdf.columns = ["Counts"]
colrefdf.reset_index(inplace=True)
colrefdf.columns = ["Ref", "Counts"]
coldiscdf = coldiscdf.merge(colrefdf, on="Ref")
coldiscdf.index = DataFrame(sColArtistToRef).index

coldiscdf = coldiscdf.merge(discdf, on=["Ref", "Name"], how='left')
coldiscdf.index = DataFrame(sColArtistToRef).index

In [ ]:
coldiscdf.to_pickle("ArtistDiscog.p")

In [ ]:
coldiscdf[coldiscdf["Name"] == "Dave Matthews"]

In [ ]:
colrefdf

In [ ]:
coldiscdf.shape

In [ ]:
coldiscdf.merge(colrefdf, on="Ref", how="left")

In [ ]:

coldiscdf.merge(sColArtistRefToCnts, on="")

In [ ]:
discdf.shape

In [ ]:
sArtistToAlbums     = Series(artistIDtoAlbumIDs)
sArtistToAlbumNames = Series(artistIDtoAlbumNames)

In [ ]:
s = Series([dict(zip(x, y)) for x,y in list(zip(sArtistToAlbums.values, sArtistToAlbumNames))], index=sArtistToAlbums.index)

In [ ]:
s

In [ ]:
fixes = {}
iDir  = None
debug = False
for ifile,iPathData in jzpathdata.items():
    iTagData = jztagdata[ifile]
    
    pathArtist = iPathData['pbArtist']
    
    maPathArtists     = ma.getArtistNames(pathArtist)
    maTagArtists      = ma.getArtistNames(iTagData['Artist'])
    maTagAlbumArtists = ma.getArtistNames(iTagData['AlbumArtist'])

    if maTagArtists == maTagAlbumArtists:
        if maTagAlbumArtists == maPathArtists:
            continue
            print("Looks good")
        else:
            if debug:
                print("\nDifferent Artists:")
                print("\tPath:  {0}".format(maPathArtists.keys()))
                print("\tTag:   {0}".format(maTagArtists.keys()))
                print("\tTagAA: {0}".format(maTagAlbumArtists.keys()))
            if iDir is None:
                iDir = ifile.split(pathArtist)[0]                
                fixes[iDir] = {}
                print("  Adding {0}".format(iDir))
            if fixes[iDir].get(pathArtist) is None:
                fixes[iDir][pathArtist] = {}
                print("\tAdding {0}".format(pathArtist))
            iData = ifile.split(pathArtist)[1][1:]
            fixes[iDir][pathArtist][iData] = {"Error": "Path", "Path": maPathArtists, "Artist": maTagArtists, "AlbumArtist": maTagAlbumArtists}
            print("\t  Adding {0}".format(iData))
    else:
        if debug:
            print("\nUnknown Artist to Discogs:")
            print("\tPath:  {0}".format(maPathArtists))
            print("\tTag:   {0}".format(maTagArtists))
            print("\tTagAA: {0}".format(maTagAlbumArtists))


        if iDir is None:
            iDir = ifile.split(pathArtist)[0]                
            fixes[iDir] = {}
            print("  Adding {0}".format(iDir))
        if fixes[iDir].get(pathArtist) is None:
            fixes[iDir][pathArtist] = {}
            print("\tAdding {0}".format(pathArtist))
        iData = ifile.split(pathArtist)[1][1:]
        fixes[iDir][pathArtist][iData] = {"Error": "Tag", "Path": maPathArtists, "Artist": maTagArtists, "AlbumArtist": maTagAlbumArtists}
        print("\t  Adding {0}".format(iData))


In [ ]:
saveFile(idata=fixes, ifile="fixes.yaml")

In [ ]:
disc = discogs()
bsdata = searchDiscogs("Duke Ellington with the Ron Collier Orchestra Collages", requireUS=True, release=True)

In [ ]:
from webUtils import getHTML
from artist import artist
#bsdata = getHTML(bsdata)

In [ ]:

h4s = bsdata.findAll("h4")
h5s = bsdata.findAll("h5")



for ih4,h4 in enumerate(h4s):
    print(h4)
    spans = h4.findAll("span")
    ref   = None
    if len(spans) == 0:
        ref = h4.find("a")
    else:
        ref = spans[0].find("a")

    if ref is None:
        continue

    try:
        href   = ref.attrs.get('href')
        name   = ref.text.strip()
    except:
        print("Could not get name/href from {0}".format(ref))
        continue


In [ ]:

ah5s

In [ ]:
artistData

In [ ]:
albumData  = [x for x in h4s if len(x.findAll('a')) > 0]
albumData  = [x.findAll('a') for x in albumData if not x.find('a').attrs['href'].startswith("/sell/")]

artistData = [x.findAll("a") for x in h5s if len(x.findAll('span')) > 0]
#artistData = [x.findAll("a") for x in artistData]

In [ ]:
                    baseURL = disc.discogURL
                    url = urllib.parse.urljoin(baseURL, href)
                    self.downloadAlbumURL(url, savename)
                

In [ ]:
def getRefName(ref):
    try:
        href   = ref.attrs.get('href')
        name   = ref.text.strip()
    except:
        href   = None
        name   = None
        
    return href,name

In [ ]:
artistData

In [ ]:
alb = album()

In [ ]:
baseURL = disc.discogURL

for (albums,artists) in zip(albumData,artistData):
    for album in albums:
        albumRef,albumName = getRefName(album)
        print(albumRef,'\t',albumName)
        try:
            albumID  = str(int(albumRef.split("/")[-1]))
        except:
            print("Could not get album ID from {0}".format(albumRef))
            albumID  = None
            
        print(albumID)
        url = urllib.parse.urljoin(baseURL, albumRef)
                
                
    for artist in artists:
        artistRef,artistName = getRefName(artist)        
        print(artistRef,'\t',artistName)
        
    

In [ ]:
album

In [ ]:
from searchUtils import findDirsPattern, findWalkExt
from fileUtils import getBaseFilename, getFileBasics
from ioUtils import saveFile, getFile
from timeUtils import clock, elapsed
from musicPath import pathBasics


##############################################################################################################################
# My Music
##############################################################################################################################
class musicFileAnalyzer(musicBase):
    def __init__(self, debug=False):
        self.name = "myfileanalyzer"
        musicBase.__init__(self, debug=debug)
        
        self.pb      = pathBasics()
        self.pathDBs = self.getMusicPathDBs()
        self.tagDBs  = self.getMusicTagDBs()

In [ ]:
mfa = musicFileAnalyzer()

In [ ]:
from fileUtils import getDirname
from collections import Counter

artistPathData = {}
albumPathData  = {}

for cls in ma.getMusicClasses():
    pathData = ma.getMusicPathDB(cls)
    tagData  = ma.getMusicTagDB(cls)
    
    start, cmt = clock("  Checking {0} mp3s".format(len(pathData)))
    for mp3,pb in pathData.items():
        
        artist = pb["pbArtist"]
        if artist is not None:
            if artistPathData.get(artist) is None:
                artistPathData[artist] = {}
        else:
            if artistPathData.get(artist) is None:
                artistPathData[artist] = {}

            
        album = pb["pbAlbum"]
        if album is not None:
            if artistPathData[artist].get(album) is None:
                artistPathData[artist][album] = {}
        else:
            if artistPathData[artist].get(album) is None:
                artistPathData[artist][album] = {}

            
        cls = pb["pbClass"]
        if album is not None:
            if artistPathData[artist][album].get(cls) is None:
                artistPathData[artist][album][cls] = {}
        else:
            if artistPathData[artist][album].get(cls) is None:
                artistPathData[artist][album][cls] = {}

                
        dirval = getDirname(mp3)
        
        if artistPathData[artist][album][cls].get(dirval) is None:
            artistPathData[artist][album][cls][dirval] = []
        artistPathData[artist][album][cls][dirval].append([pb, tagData[mp3]])
    elapsed(start, cmt)
    print("")
    
    
savename = setFile(ma.getDBDir(), "ArtistAlbumTagData.p")
saveFile(ifile=savename, idata=artistPathData, debug=True)


In [ ]:
savename = setFile(ma.getDBDir(), "ArtistAlbumTagData.p")
artistPathData = getFile(savename)

In [ ]:
nArt = 0
for artist,artistData in artistPathData.items():
    nArt += 1
    if nArt == 10:
        break
    for album,albumData in artistData.items():
        for cls,clsData in albumData.items():
            for dirval,dirvalData in clsData.items():
                
                dval = dirval.replace(ma.getMusicDir(), "")
                
                nFiles = len(dirvalData)
                partist = artist
                if len(artist) >= 40:
                    partist = "{0}...".format(artist[:36])
                palbum = album
                if len(album) >= 40:
                    palbum = "{0}...".format(album[:36])

                print("{0: <40}{1: <40}{2: <20}{3: <5}{4}".format(partist,palbum,cls,nFiles,dirval))

In [ ]:
artistPathData

In [ ]:
for ifile in ma.flists:
    tagfile  = ifile.replace(".p", "-Tags.p")
    pathfile = ifile.replace(".p", "-Paths.p")

    fileData = getFile(ifile)
    tagData  = getFile(tagfile)
    pathData = getFile(pathfile)

In [ ]:
from searchUtils import findExt
files = findExt(basedir=ma.getDBDir(), ext="*.p")
files = [x for x in files if sum([y in x for y in ["Tags.p", "Paths.p"]]) == 0]
files

In [ ]:
import os, sys
def splitall(path):
    allparts = []
    while 1:
        parts = os.path.split(path)
        if parts[0] == path:  # sentinel for absolute paths
            allparts.insert(0, parts[0])
            break
        elif parts[1] == path: # sentinel for relative paths
            allparts.insert(0, parts[1])
            break
        else:
            path = parts[0]
            allparts.insert(0, parts[1])
    return allparts

In [ ]:
os.path.split("24 Shots/01 Intro (Power Of The Dollar)/dfd")

In [ ]:
idir

In [ ]:
files[-2]

In [ ]:
ifile

In [ ]:
filename

In [ ]:
dirval

In [ ]:
artist

In [ ]:
%load_ext autoreload
%autoreload
disc = discogs()

In [ ]:
import argparse as ap
from os import getcwd
from mp3id import mp3ID
mid = mp3ID(ifile)
minfo = mid.getInfo()
from glob import glob
from fsUtils import isFile
from fileUtils import getExt
from searchUtils import findWalkExt, findWalkPattern
from os import listdir, walk

class mp3Tagger:
    def __init__(self, disc=None):
        self.name = "mp3tagger"
        self.cwd  = getcwd()
        
        self.infos = {}
        self.ma       = multiArtist(cutoff=0.9, discdata=None, exact=False)

        
        if disc is not None:
            self.setDiscogs(disc)
        
        self.exts = mp3ID().mp3exts
        

    def setDiscogs(self, disc):
        artistDB = disc.getArtistVariationNameToIDsData()
        self.setArtistsDB(artistDB=artistDB)
        
    def setArtistsDB(self, artistDB, cutoff=0.9):
        self.ma       = multiArtist(cutoff=0.9, discdata=artistDB, exact=False)
        
        
    def getFiles(self, dirval):
        self.infos = {}
        files = findWalkPattern(basedir=dirval, pattern=".")
        files = [x for x in files if getExt(x) in self.exts]
        for i,ifile in enumerate(files):
            if self.infos.get(ifile) is None:
                mid = mp3ID(ifile)
                minfo = mid.getInfo()
                self.infos[ifile] = minfo
                
        
    def get(self):
        return self.infos
        
    def getInfo(self, dirval=None):
        if dirval is not None:
            self.getFiles(dirval)
            
        print("{0: <3}{1: <4}{2: <4}{3: <50}{4: <50}{5: <40}{6: <40}".format("#", "Trk", "Dsc", "Title", "Album", "Artist", "Album Artist"))

        i = 0
        for ifile,minfo in self.infos.items():
            print("{0: <3}".format(i+1), end="")

            trkno = minfo.get('TrackNo')
            if trkno is None:
                trkno = -1
            elif isinstance(trkno, list):
                if len(trkno) == 1:
                    trkno = trkno[0]
                else:
                    trkno = str(trkno)
            print("{0: <4}".format(trkno), end="")

            discno = minfo.get('DiscNo')
            if discno is None:
                discno = -1
            elif isinstance(discno, list):
                if len(discno) == 1:
                    discno = discno[0]
                else:
                    discno = str(discno)                
            print("{0: <4}".format(discno), end="")

            title = minfo.get('Title')
            if title is None:
                title = "?"
            elif isinstance(title, list):
                if len(title) == 1:
                    title = title[0]
                else:
                    title = str(title)
            print("{0: <50}".format(title), end="")

            album = minfo.get('Album')
            if album is None:
                album = "?"
            elif isinstance(album, list):
                if len(album) == 1:
                    album = album[0]
                else:
                    album = str(album)
            print("{0: <50}".format(album), end="")

            artist = minfo.get('Artist')
            if artist is None:
                artist = "?"
            elif isinstance(artist, list):
                if len(artist) == 1:
                    artist = artist[0]
                else:
                    artist = str(artist)
            print("{0: <40}".format(artist), end="")

            albartist = minfo.get('AlbumArtist')
            if albartist is None:
                albartist = "?"
            elif isinstance(albartist, list):
                if len(albartist) == 1:
                    albartist = albartist[0]
                else:
                    albartist = str(albartist)
            print("{0: <40}".format(albartist), end="")
            print("")
            
            i += 1
            
    

    def getAlbumArtists(self):
        return [x.get("AlbumArtist") for ifile,x in self.infos.items()]

In [ ]:
mt = mp3Tagger(disc=disc)

In [ ]:
mt.getInfo(dirval="/Volumes/Music/iTunes Jazz/iTunes Media/Music/Miles Davis & Quincy Jones")

In [ ]:
from ioUtils import saveFile
retvals = {}
for ifile,minfo in mt.get().items():
    retvals[ifile] = {}

    for key in ["Artist", "AlbumArtist"]:
        val = minfo.get(key)
        if isinstance(val, list):
            if len(val) == 1:
                retval = mt.ma.getArtistNames(val[0])
                if len(retval) > 1:
                    artistNames = [k for k,v in retval.items() if v is not None]
                    retvals[ifile][key] = artistNames
                    
saveFile(idata=retvals, ifile="tmp.yaml")

In [ ]:
retvals

In [ ]:
tmp=[['Miles Davis & Quincy Jones'],
 ['Miles Davis & Quincy Jones'],
 ['Miles Davis & Quincy Jones'],
 ['Miles Davis & Quincy Jones'],
 ['Miles Davis & Quincy Jones'],
 ['Miles Davis & Quincy Jones'],
 ['Miles Davis & Quincy Jones'],
 ['Miles Davis & Quincy Jones'],
 ['Miles Davis & Quincy Jones'],
 ['Miles Davis & Quincy Jones'],
 ['Miles Davis & Quincy Jones'],
 ['Miles Davis & Quincy Jones'],
 ['Miles Davis & Quincy Jones'],
 ['Miles Davis & Quincy Jones'],
 ['Miles Davis & Quincy Jones'],
 ['Miles Davis & Quincy Jones']]

In [ ]:
m.getDBDir

In [ ]:
idir="/Volumes/Music/iTunes Jazz/iTunes Media/Music/Miles Davis & Quincy Jones


from random import shuffle
print("There are {0} chart artists".format(len(chartArtists)))
shuffle(chartArtists)
for i,artist in enumerate(chartArtists):
    if prevs.get(artist) is not None:
        continue
    retval = ma.getArtistNames(artist)